In [2]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [5]:
input_size = 784  # Pixels
hidden_size = 400 # input + output / 2
out_size = 10
epochs = 10
batch_size = 100
learning_rate = 0.001

In [6]:
train_dataset = datasets.MNIST(root='./data2',
                              train=True,
                              transform=transforms.ToTensor(),
                              download=False)

test_dataset = datasets.MNIST(root='./data2',
                             train=False,
                             transform=transforms.ToTensor())

In [7]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                         batch_size=batch_size,
                                         shuffle=False)

In [19]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, out_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.batchnorm = nn.BatchNorm1d(hidden_size)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool1d(kernel_size=2)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, out_size)
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.batchnorm(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out

In [20]:
net = Net(input_size, hidden_size, out_size)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

In [21]:
net.parameters

<bound method Module.parameters of Net(
  (fc1): Linear(in_features=784, out_features=400, bias=True)
  (batchnorm): BatchNorm1d(400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (maxpool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc2): Linear(in_features=400, out_features=400, bias=True)
  (fc3): Linear(in_features=400, out_features=10, bias=True)
)>

In [ ]:
# Training the network
correct_train = 0
total_train = 0
for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1, 28*28))
        labels = Variable(labels)
        
        optimizer.zero_grad()
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum()
        
        loss = criterion(outputs, labels)
        loss.backward()
        
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print(f"Epoch [{epoch+1}/{epochs}], Iteration [{i+1}/{len(train_dataset)//batch_size}], Training Loss: {loss.data}, Training Accuracy: {100*correct_train/total_train}%")
print('DONE TRAINING')

Epoch [1/10], Iteration [100/600], Training Loss: 0.14155535399913788, Training Accuracy: 87.0199966430664%
Epoch [1/10], Iteration [200/600], Training Loss: 0.19047187268733978, Training Accuracy: 90.19499969482422%
Epoch [1/10], Iteration [300/600], Training Loss: 0.13459190726280212, Training Accuracy: 91.71333312988281%
Epoch [1/10], Iteration [400/600], Training Loss: 0.1428428292274475, Training Accuracy: 92.61750030517578%
Epoch [1/10], Iteration [500/600], Training Loss: 0.09305428713560104, Training Accuracy: 93.2979965209961%
Epoch [1/10], Iteration [600/600], Training Loss: 0.12881779670715332, Training Accuracy: 93.81999969482422%
Epoch [2/10], Iteration [100/600], Training Loss: 0.02476913295686245, Training Accuracy: 94.3414306640625%
Epoch [2/10], Iteration [200/600], Training Loss: 0.13356433808803558, Training Accuracy: 94.69000244140625%
Epoch [2/10], Iteration [300/600], Training Loss: 0.05044255405664444, Training Accuracy: 94.96221923828125%
Epoch [2/10], Iteration

In [18]:
# Testing the network
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, 28*28))
    
    outputs = net(images)
    
    _, predicted = torch.max(outputs.data, 1)
    
    total += labels.size(0)
    correct += (predicted == labels).sum()
    
print(f"Test Accuracy: {100*correct/total} %")

Test Accuracy: 97.37999725341797 %
